<a href="https://colab.research.google.com/github/Trading-com-Dados/desenvolvimento_interno/blob/main/20220413_Backtesting_a_mao_monitoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **BACKTESTING  "*à mão*": o céu é o limite!** 
---
#### *Disclaimer:*
#### Este código foi elaborado para fins exclusivamente educacionais. O conteúdo aqui apresentado visa fornecer informações complementares para auxiliar o investidor na tomada de suas próprias decisões de investimento. Reforçando, nenhum tópico aqui abordado constitui qualquer tipo de indicação/oferta/solicitação de compra/venda de qualquer produto.
---



<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300">

# **1. Bibliotecas utilizadas**

In [6]:
!pip install yfinance
!pip install python-binance
!pip install ta

     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 10.9 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████████████| 111 kB 52.5 MB/s 
     |████████████████████████████████| 288 kB 56.3 MB/s 
     |████████████████████████████████| 271 kB 56.0 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 62.1 MB/s 


In [70]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz

import yfinance as yf
from binance.client import Client
import ta

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# **2. Definir o setup e baixar os dados**

## 2.1. Definir sinais de Entrada 

---
**OBS.: Durante a aula do dia 13/04/22 acabei me confundindo e misturando os dois setups abaixo**

---

**Preço de fechamento de reversão**

Sinal de Entrada:<p>
1) Quando a mínima do candle atual for menor do que a mínima dos dois últimos candles prévios e; <p>
2) o fechamento do candle atual for maior do que o fechamento do candle anterior.

**123 de compra simples**

Sinal de Entrada:<p>
1) Um candle que tenha a mínima menor que a mínima do candle anterior e; <p>
2) Vamos comprar na superação da máxima desse candle formando os três candles e, <p>
3) Isso em cima da MM9 ascendente.

---
**Mas vamos aproveitar e colocar os dois em prática para comparar os resultados**

---

## 2.2. Baixar os dados (yfinance ou binance)

In [35]:
# Os conjuntos de dados das bibliotecas são padronizados em UTC (horário padrão global =  Brasília - 3h)

hora_agora_BR = datetime.now(pytz.timezone("America/Sao_Paulo"))
hora_agora_BR

datetime.datetime(2022, 4, 13, 22, 51, 25, 989934, tzinfo=<DstTzInfo 'America/Sao_Paulo' -03-1 day, 21:00:00 STD>)

In [37]:
# Acho que Bitcoin pelo Yahoo Finance só em dolar mesmo
# Dependendo do horário em que o código for rodado, os candles já corresponderão ao dia de amanhã

cotacoes_BTC_yf = yf.download("BTC-USD", start = '2018-04-01', interval = "1D")
cotacoes_BTC_yf

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-04-01,7003.060059,7060.950195,6526.870117,6844.229980,6844.229980,4532100096
2018-04-02,6844.859863,7135.470215,6816.580078,7083.799805,7083.799805,4333440000
2018-04-03,7102.259766,7530.939941,7072.490234,7456.109863,7456.109863,5499700224
2018-04-04,7456.410156,7469.879883,6803.879883,6853.839844,6853.839844,4936000000
2018-04-05,6848.649902,6933.819824,6644.799805,6811.470215,6811.470215,5639320064
...,...,...,...,...,...,...
2022-04-09,42282.078125,42786.816406,42183.253906,42782.136719,42782.136719,16050772496
2022-04-10,42781.089844,43376.378906,42021.207031,42207.671875,42207.671875,17654475582
2022-04-11,42201.039062,42424.589844,39373.058594,39521.902344,39521.902344,33949912166


In [38]:
# Utilizando a python-binance (possui BTC em BRL)
# Um pouco mais complexo o início para produzir o dataframe final

api_secret = 'nao_necessario_para_cotacoes'
api_key = 'apenas_para_trading_real'

In [39]:
end_str = datetime.now(pytz.timezone("UTC")).strftime("%Y-%m-%d")
end_str

'2022-04-14'

In [47]:
client = Client()
cotacoes_BTC_BRL = client.get_historical_klines(symbol='BTCBRL', start_str = '2018-04-01', end_str = end_str, interval=Client.KLINE_INTERVAL_1DAY)
cotacoes_BTC_BRL[0:2]

[[1602547200000,
  '63606.00000000',
  '64059.00000000',
  '46208.00000000',
  '63921.00000000',
  '2.87607300',
  1602633599999,
  '181785.88362100',
  370,
  '2.32937400',
  '147274.38088300',
  '0'],
 [1602633600000,
  '63920.00000000',
  '64314.00000000',
  '62677.00000000',
  '64068.00000000',
  '7.02546000',
  1602719999999,
  '447815.98757600',
  519,
  '4.47722500',
  '285514.69429100',
  '0']]

In [41]:
BTC_BRL_df = pd.DataFrame(cotacoes_BTC_BRL)
BTC_BRL_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1602547200000,63606.00000000,64059.00000000,46208.00000000,63921.00000000,2.87607300,1602633599999,181785.88362100,370,2.32937400,147274.38088300,0
1,1602633600000,63920.00000000,64314.00000000,62677.00000000,64068.00000000,7.02546000,1602719999999,447815.98757600,519,4.47722500,285514.69429100,0
2,1602720000000,64068.00000000,65764.00000000,63505.00000000,65314.00000000,10.53093500,1602806399999,682399.26519400,498,9.31532100,603488.37039800,0
3,1602806400000,65334.00000000,65560.00000000,63650.00000000,64750.00000000,9.18102600,1602892799999,593389.07995100,447,5.74067700,371335.12349600,0
4,1602892800000,64600.00000000,65025.00000000,64418.00000000,64665.00000000,1.89171500,1602979199999,122331.44754300,101,1.06647100,69085.62068600,0
...,...,...,...,...,...,...,...,...,...,...,...,...
544,1649548800000,202979.00000000,205920.00000000,198599.00000000,200204.00000000,166.81185000,1649635199999,33818456.85436000,12798,75.56504000,15330424.59228000,0
545,1649635200000,200065.00000000,201081.00000000,185915.00000000,187418.00000000,498.96877000,1649721599999,96521904.97028000,34941,219.18860000,42424712.97338000,0
546,1649721600000,187441.00000000,192407.00000000,185156.00000000,188910.00000000,470.54003000,1649807999999,88745371.73495000,26302,229.59564000,43320981.79753000,0
547,1649808000000,189000.00000000,195770.00000000,186200.00000000,194543.00000000,384.42073000,1649894399999,73537558.90339000,19572,189.66631000,36329424.24103000,0


Lista das colunas obtidas (Open time, Open, High, Low, Close, Volume, Close time, Quote asset volume, Number of trades, Taker buy base asset volume, Taker buy quote asset volume, Ignore)

In [42]:
list(enumerate(['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']))

[(0, 'Open time'),
 (1, 'Open'),
 (2, 'High'),
 (3, 'Low'),
 (4, 'Close'),
 (5, 'Volume'),
 (6, 'Close time'),
 (7, 'Quote asset volume'),
 (8, 'Number of trades'),
 (9, 'Taker buy base asset volume'),
 (10, 'Taker buy quote asset volume'),
 (11, 'Ignore')]

In [43]:
BTC_BRL_df.rename(columns={0:'Date',1:'Open',2:'High',3:'Low',4:'Close',5:'Volume'},inplace=True)
BTC_BRL_df.set_index('Date', inplace=True)
BTC_BRL_df

,Open,High,Low,Close,Volume,6,7,8,9,10,11
Date,,,,,,,,,,,
1602547200000,63606.00000000,64059.00000000,46208.00000000,63921.00000000,2.87607300,1602633599999,181785.88362100,370,2.32937400,147274.38088300,0
1602633600000,63920.00000000,64314.00000000,62677.00000000,64068.00000000,7.02546000,1602719999999,447815.98757600,519,4.47722500,285514.69429100,0
1602720000000,64068.00000000,65764.00000000,63505.00000000,65314.00000000,10.53093500,1602806399999,682399.26519400,498,9.31532100,603488.37039800,0
1602806400000,65334.00000000,65560.00000000,63650.00000000,64750.00000000,9.18102600,1602892799999,593389.07995100,447,5.74067700,371335.12349600,0
1602892800000,64600.00000000,65025.00000000,64418.00000000,64665.00000000,1.89171500,1602979199999,122331.44754300,101,1.06647100,69085.62068600,0
...,...,...,...,...,...,...,...,...,...,...,...
1649548800000,202979.00000000,205920.00000000,198599.00000000,200204.00000000,166.81185000,1649635199999,33818456.85436000,12798,75.56504000,15330424.59228000,0
1649635200000,200065.00000000,201081.00000000,185915.00000000,187418.00000000,498.96877000,1649721599999,96521904.97028000,34941,219.18860000,42424712.97338000,0
1649721600000,187441.00000000,192407.00000000,185156.00000000,188910.00000000,470.54003000,1649807999999,88745371.73495000,26302,229.59564000,43320981.79753000,0


In [44]:
BTC_BRL_df.index = pd.to_datetime(BTC_BRL_df.index,unit='ms')
BTC_BRL_df = BTC_BRL_df.drop(BTC_BRL_df.columns[5:],axis=1)
BTC_BRL_df

,Open,High,Low,Close,Volume
Date,,,,,
2020-10-13,63606.00000000,64059.00000000,46208.00000000,63921.00000000,2.87607300
2020-10-14,63920.00000000,64314.00000000,62677.00000000,64068.00000000,7.02546000
2020-10-15,64068.00000000,65764.00000000,63505.00000000,65314.00000000,10.53093500
2020-10-16,65334.00000000,65560.00000000,63650.00000000,64750.00000000,9.18102600
2020-10-17,64600.00000000,65025.00000000,64418.00000000,64665.00000000,1.89171500
...,...,...,...,...,...
2022-04-10,202979.00000000,205920.00000000,198599.00000000,200204.00000000,166.81185000
2022-04-11,200065.00000000,201081.00000000,185915.00000000,187418.00000000,498.96877000
2022-04-12,187441.00000000,192407.00000000,185156.00000000,188910.00000000,470.54003000


In [71]:
BTC_BRL_df['Open'] = pd.to_numeric(BTC_BRL_df['Open'])
BTC_BRL_df['High'] = pd.to_numeric(BTC_BRL_df['High'])
BTC_BRL_df['Low'] = pd.to_numeric(BTC_BRL_df['Low'])
BTC_BRL_df['Close'] = pd.to_numeric(BTC_BRL_df['Close'])
BTC_BRL_df['Volume'] = pd.to_numeric(BTC_BRL_df['Volume'])
BTC_BRL_df

,Open,High,Low,Close,Volume
Date,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460
2020-10-15,64068.0,65764.0,63505.0,65314.0,10.530935
2020-10-16,65334.0,65560.0,63650.0,64750.0,9.181026
2020-10-17,64600.0,65025.0,64418.0,64665.0,1.891715
...,...,...,...,...,...
2022-04-10,202979.0,205920.0,198599.0,200204.0,166.811850
2022-04-11,200065.0,201081.0,185915.0,187418.0,498.968770
2022-04-12,187441.0,192407.0,185156.0,188910.0,470.540030


---
**PAUSA (INICIO): Duvida durante plantão: Teste volume ao longo do dia das criptos**

---



In [72]:
client = Client()
btc_df = client.get_historical_klines(symbol='BTCBRL', start_str = '2022-03-01', end_str = end_str, interval=Client.KLINE_INTERVAL_30MINUTE)

btc_df = pd.DataFrame(btc_df)
btc_df.rename(columns={0:'Open time',1:'open',2:'high',3:'low',4:'close',5:'volume',6:'Close time'},inplace=True)
#btc_df.set_index('date', inplace=True)
btc_df['Open time'] = pd.to_datetime(btc_df['Open time'],unit='ms')
btc_df['Close time'] = pd.to_datetime(btc_df['Close time'],unit='ms')
btc_df = btc_df.drop(btc_df.columns[7:],axis=1)
btc_df['open'] = pd.to_numeric(btc_df['open'])
btc_df['high'] = pd.to_numeric(btc_df['high'])
btc_df['low'] = pd.to_numeric(btc_df['low'])
btc_df['close'] = pd.to_numeric(btc_df['close'])
btc_df['volume'] = pd.to_numeric(btc_df['volume'])
btc_df

,Open time,open,high,low,close,volume,Close time
0,2022-03-01 00:00:00,222697.0,224857.0,222652.0,223617.0,8.01033,2022-03-01 00:29:59.999
1,2022-03-01 00:30:00,223567.0,225343.0,223365.0,224649.0,11.92042,2022-03-01 00:59:59.999
2,2022-03-01 01:00:00,224615.0,224825.0,222637.0,222843.0,6.36851,2022-03-01 01:29:59.999
3,2022-03-01 01:30:00,222907.0,223686.0,222692.0,222770.0,8.26647,2022-03-01 01:59:59.999
4,2022-03-01 02:00:00,222775.0,222998.0,221368.0,221703.0,7.73150,2022-03-01 02:29:59.999
...,...,...,...,...,...,...,...
2108,2022-04-13 22:00:00,194917.0,195330.0,194648.0,195089.0,3.28894,2022-04-13 22:29:59.999
2109,2022-04-13 22:30:00,195225.0,195577.0,194650.0,194959.0,3.71596,2022-04-13 22:59:59.999
2110,2022-04-13 23:00:00,195031.0,195180.0,194050.0,194446.0,6.09422,2022-04-13 23:29:59.999
2111,2022-04-13 23:30:00,194416.0,194761.0,194200.0,194543.0,2.37627,2022-04-13 23:59:59.999


In [73]:
btc_df['Hora'] = pd.to_datetime(btc_df['Close time']).dt.strftime('%H')
btc_df['Dia_mes'] = pd.to_datetime(btc_df['Close time']).dt.strftime('%m-%d')
btc_df

,Open time,open,high,low,close,volume,Close time,Hora,Dia_mes
0,2022-03-01 00:00:00,222697.0,224857.0,222652.0,223617.0,8.01033,2022-03-01 00:29:59.999,00,03-01
1,2022-03-01 00:30:00,223567.0,225343.0,223365.0,224649.0,11.92042,2022-03-01 00:59:59.999,00,03-01
2,2022-03-01 01:00:00,224615.0,224825.0,222637.0,222843.0,6.36851,2022-03-01 01:29:59.999,01,03-01
3,2022-03-01 01:30:00,222907.0,223686.0,222692.0,222770.0,8.26647,2022-03-01 01:59:59.999,01,03-01
4,2022-03-01 02:00:00,222775.0,222998.0,221368.0,221703.0,7.73150,2022-03-01 02:29:59.999,02,03-01
...,...,...,...,...,...,...,...,...,...
2108,2022-04-13 22:00:00,194917.0,195330.0,194648.0,195089.0,3.28894,2022-04-13 22:29:59.999,22,04-13
2109,2022-04-13 22:30:00,195225.0,195577.0,194650.0,194959.0,3.71596,2022-04-13 22:59:59.999,22,04-13
2110,2022-04-13 23:00:00,195031.0,195180.0,194050.0,194446.0,6.09422,2022-04-13 23:29:59.999,23,04-13
2111,2022-04-13 23:30:00,194416.0,194761.0,194200.0,194543.0,2.37627,2022-04-13 23:59:59.999,23,04-13


In [74]:
volume_horas_bitcoin = btc_df.pivot_table(values='volume',index='Hora',columns=[],aggfunc=np.sum)
volume_horas_bitcoin

,volume
Hora,
00,652.80632
01,671.48896
02,568.06708
03,449.87732
04,431.68127
05,298.08396
06,253.88052
07,261.85531
08,313.85691


In [80]:
fig = px.histogram(volume_horas_bitcoin, x=volume_horas_bitcoin.index,y='volume')
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Volume ao longo das horas (lembrando que está em UTC)',width=800 ,height=800)
fig.show()

---
**PAUSA (FIM): Duvida durante plantão: Teste volume ao longo do dia das criptos**

---

# **3. Backtesting Setup Preço de fechamento de reversão (PFR)**

* Bitcoin BRL<p>
* Timeframe diário
* Entre 2018 e o dia de hoje (ver o dia)

In [106]:
BTC_BRL_df

,Open,High,Low,Close,Volume
Date,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460
2020-10-15,64068.0,65764.0,63505.0,65314.0,10.530935
2020-10-16,65334.0,65560.0,63650.0,64750.0,9.181026
2020-10-17,64600.0,65025.0,64418.0,64665.0,1.891715
...,...,...,...,...,...
2022-04-10,202979.0,205920.0,198599.0,200204.0,166.811850
2022-04-11,200065.0,201081.0,185915.0,187418.0,498.968770
2022-04-12,187441.0,192407.0,185156.0,188910.0,470.540030


**Preço de fechamento de reversão**

Sinal de Entrada:<p>
1) Quando a mínima do candle atual for menor do que a mínima dos dois últimos candles prévios e; <p>
2) O fechamento do candle atual for maior do que o fechamento do candle anterior.

**Regra 1:** Low2 < (Low1 e Low0)

**Regra 2:** Close2 > Close1

## **3.1. Sinais de entrada COMPRA e ordem Buy Limit**


**Regra 1:** Low2 < (Low1 e Low0)

In [85]:
# closed = 'left' exclui o próprio valor

min_2cd_ant = BTC_BRL_df['Low'].rolling(window=2,closed='left').min()
min_2cd_ant

Date
2020-10-13         NaN
2020-10-14         NaN
2020-10-15     46208.0
2020-10-16     62677.0
2020-10-17     63505.0
                ...   
2022-04-10    199846.0
2022-04-11    198599.0
2022-04-12    185915.0
2022-04-13    185156.0
2022-04-14    185156.0
Name: Low, Length: 549, dtype: float64

---
**PAUSA (INICIO): Para entender melhor o parâmetro 'closed' de rolling**

---

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

In [100]:
df_teste = pd.DataFrame({'A':list(range(0,5))})
df_teste

,A
0,0
1,1
2,2
3,3
4,4


In [101]:
# Left "exclui o valor dela mesma"
df_teste['min_left'] = df_teste['A'].rolling(window=1,closed='left').min()
df_teste['max_left'] = df_teste['A'].rolling(window=1,closed='left').max()
df_teste

,A,min_left,max_left
0,0,NaN,NaN
1,1,0.0,0.0
2,2,1.0,1.0
3,3,2.0,2.0
4,4,3.0,3.0


In [102]:
# Left "exclui o valor dela mesma"
df_teste['min_left'] = df_teste['A'].rolling(window=2,closed='left').min()
df_teste['max_left'] = df_teste['A'].rolling(window=2,closed='left').max()
df_teste

,A,min_left,max_left
0,0,NaN,NaN
1,1,NaN,NaN
2,2,0.0,1.0
3,3,1.0,2.0
4,4,2.0,3.0


In [103]:
# Right inclui o valor dela mesma mas exclui o que seria o primeiro

df_teste['min_left'] = df_teste['A'].rolling(window=1,closed='right').min()
df_teste['max_left'] = df_teste['A'].rolling(window=1,closed='right').max()
df_teste

,A,min_left,max_left
0,0,0.0,0.0
1,1,1.0,1.0
2,2,2.0,2.0
3,3,3.0,3.0
4,4,4.0,4.0


In [104]:
# Right inclui o valor dela mesma mas exclui o que seria o primeiro

df_teste['min_left'] = df_teste['A'].rolling(window=2,closed='right').min()
df_teste['max_left'] = df_teste['A'].rolling(window=2,closed='right').max()
df_teste

,A,min_left,max_left
0,0,NaN,NaN
1,1,0.0,1.0
2,2,1.0,2.0
3,3,2.0,3.0
4,4,3.0,4.0


---
**PAUSA (FIM): Para entender melhor o parâmetro 'closed' de rolling**

---

In [141]:
## Várias formas de analisar as mínimas dos últimos dois candles (sem considerar o próprio candle)

min_2cd_ant = BTC_BRL_df['Low'].rolling(window=2,closed='left').min()
BTC_BRL_df['min_2cd_ant'] = min_2cd_ant
BTC_BRL_df.head()

,Open,High,Low,Close,Volume,min_2cd_ant,min_2cd_ant_shift,min_2cd_ant_iloc,Equal_1_2
Date,,,,,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073,NaN,NaN,NA,False
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460,NaN,NaN,NA,False
2020-10-15,64068.0,65764.0,63505.0,65314.0,10.530935,46208.0,46208.0,46208.0,True
2020-10-16,65334.0,65560.0,63650.0,64750.0,9.181026,62677.0,62677.0,62677.0,True
2020-10-17,64600.0,65025.0,64418.0,64665.0,1.891715,63505.0,63505.0,63505.0,True


In [164]:
BTC_BRL_df['Low'].shift(2)[2]

46208.0

In [173]:
## Mínima dos últimos 2 candles (outra forma)

list_min_2cd_ant_shift = ["NA"]

for i in range(0,len(BTC_BRL_df)-1):
  min_2cd_ant_shift = min( BTC_BRL_df['Low'].shift(1)[i], BTC_BRL_df['Low'].shift(2)[i])
  list_min_2cd_ant_shift.append(min_2cd_ant_shift)

In [174]:
len(list_min_2cd_ant_shift)

549

In [175]:
## Mínima dos últimos 2 candles (mais uma alternativa usando o iloc)

lista_min_2cd_ant_iloc = ["NA","NA"]

for i in range(0,len(BTC_BRL_df)-2):
  min_2cd_ant_iloc = min( BTC_BRL_df['Low'].iloc[i-len(BTC_BRL_df)], BTC_BRL_df['Low'].iloc[i-len(BTC_BRL_df)+1] )
  lista_min_2cd_ant_iloc.append(min_2cd_ant_iloc)

In [176]:
lista_min_2cd_ant_iloc[0:10]

['NA',
 'NA',
 46208.0,
 62677.0,
 63505.0,
 63650.0,
 64418.0,
 64867.0,
 65112.0,
 66398.0]

In [177]:
print(len(BTC_BRL_df))
print(len(lista_min_2cd_ant_iloc))

549
549


In [178]:
BTC_BRL_df['min_2cd_ant_iloc'] = lista_min_2cd_ant_iloc
BTC_BRL_df

,Open,High,Low,Close,Volume,min_2cd_ant,min_2cd_ant_shift,min_2cd_ant_iloc,Equal_1_2,Equal_1_3,Equal_2_3
Date,,,,,,,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073,NaN,NaN,NA,False,False,False
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460,NaN,NaN,NA,False,False,False
2020-10-15,64068.0,65764.0,63505.0,65314.0,10.530935,46208.0,46208.0,46208.0,True,True,True
2020-10-16,65334.0,65560.0,63650.0,64750.0,9.181026,62677.0,62677.0,62677.0,True,True,True
2020-10-17,64600.0,65025.0,64418.0,64665.0,1.891715,63505.0,63505.0,63505.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-10,202979.0,205920.0,198599.0,200204.0,166.811850,199846.0,199846.0,199846.0,True,True,True
2022-04-11,200065.0,201081.0,185915.0,187418.0,498.968770,198599.0,199860.0,198599.0,False,True,False
2022-04-12,187441.0,192407.0,185156.0,188910.0,470.540030,185915.0,198599.0,185915.0,False,True,False


In [179]:
# Checar se todas as três colunas são idênticas

BTC_BRL_df['min_2cd_ant_shift'].equals(BTC_BRL_df['min_2cd_ant_iloc'])

#BTC_BRL_df['min_2cd_ant'] == BTC_BRL_df['min_2cd_ant_shift'] == BTC_BRL_df['min_2cd_ant_iloc']

False

In [180]:
BTC_BRL_df["Equal_1_2"] = BTC_BRL_df['min_2cd_ant'] == BTC_BRL_df['min_2cd_ant_shift']
BTC_BRL_df["Equal_1_3"] = BTC_BRL_df['min_2cd_ant'] == BTC_BRL_df['min_2cd_ant_iloc']
BTC_BRL_df["Equal_2_3"] = BTC_BRL_df['min_2cd_ant_shift'] == BTC_BRL_df['min_2cd_ant_iloc']
BTC_BRL_df

,Open,High,Low,Close,Volume,min_2cd_ant,min_2cd_ant_shift,min_2cd_ant_iloc,Equal_1_2,Equal_1_3,Equal_2_3
Date,,,,,,,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073,NaN,NaN,NA,False,False,False
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460,NaN,NaN,NA,False,False,False
2020-10-15,64068.0,65764.0,63505.0,65314.0,10.530935,46208.0,46208.0,46208.0,True,True,True
2020-10-16,65334.0,65560.0,63650.0,64750.0,9.181026,62677.0,62677.0,62677.0,True,True,True
2020-10-17,64600.0,65025.0,64418.0,64665.0,1.891715,63505.0,63505.0,63505.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-10,202979.0,205920.0,198599.0,200204.0,166.811850,199846.0,199846.0,199846.0,True,True,True
2022-04-11,200065.0,201081.0,185915.0,187418.0,498.968770,198599.0,199860.0,198599.0,False,True,False
2022-04-12,187441.0,192407.0,185156.0,188910.0,470.540030,185915.0,198599.0,185915.0,False,True,False


In [181]:
BTC_BRL_df[BTC_BRL_df['Equal_1_3']!=True]

,Open,High,Low,Close,Volume,min_2cd_ant,min_2cd_ant_shift,min_2cd_ant_iloc,Equal_1_2,Equal_1_3,Equal_2_3
Date,,,,,,,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073,NaN,NaN,NA,False,False,False
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460,NaN,NaN,NA,False,False,False


In [182]:
BTC_BRL_df[BTC_BRL_df['Equal_1_2']!=True]

,Open,High,Low,Close,Volume,min_2cd_ant,min_2cd_ant_shift,min_2cd_ant_iloc,Equal_1_2,Equal_1_3,Equal_2_3
Date,,,,,,,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073,NaN,NaN,NA,False,False,False
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460,NaN,NaN,NA,False,False,False
2020-10-27,74348.0,79610.0,74324.0,78649.0,28.274538,72850.0,73230.0,72850.0,False,True,False
2020-11-03,78865.0,81769.0,77296.0,81400.0,23.562322,76771.0,79308.0,76771.0,False,True,False
2020-11-08,80907.0,85334.0,80295.0,84488.0,25.459966,78164.0,83145.0,78164.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-08,208145.0,210584.0,199846.0,200402.0,417.736820,203334.0,205000.0,203334.0,False,True,False
2022-04-09,200332.0,203045.0,199860.0,202899.0,140.514090,199846.0,203334.0,199846.0,False,True,False
2022-04-11,200065.0,201081.0,185915.0,187418.0,498.968770,198599.0,199860.0,198599.0,False,True,False


In [154]:
BTC_BRL_df.head(50)

,Open,High,Low,Close,Volume,min_2cd_ant,min_2cd_ant_shift,min_2cd_ant_iloc,Equal_1_2,Equal_1_3,Equal_2_3
Date,,,,,,,,,,,
2020-10-13,63606.0,64059.0,46208.0,63921.0,2.876073,NaN,NaN,NA,False,False,False
2020-10-14,63920.0,64314.0,62677.0,64068.0,7.025460,NaN,NaN,NA,False,False,False
2020-10-15,64068.0,65764.0,63505.0,65314.0,10.530935,46208.0,46208.0,46208.0,True,True,True
2020-10-16,65334.0,65560.0,63650.0,64750.0,9.181026,62677.0,62677.0,62677.0,True,True,True
2020-10-17,64600.0,65025.0,64418.0,64665.0,1.891715,63505.0,63505.0,63505.0,True,True,True
2020-10-18,64867.0,65667.0,64867.0,65667.0,5.881325,63650.0,63650.0,63650.0,True,True,True
2020-10-19,65772.0,66865.0,65112.0,66642.0,13.220203,64418.0,64418.0,64418.0,True,True,True
2020-10-20,66703.0,67873.0,66398.0,67574.0,12.395854,64867.0,64867.0,64867.0,True,True,True
2020-10-21,67585.0,74836.0,67542.0,72600.0,46.838386,65112.0,65112.0,65112.0,True,True,True


In [ ]:
min_2cd_ant_iloc = BTC_BRL_df['Low'].shift(2)
BTC_BRL_df['min_2cd_ant_iloc'] = min_2cd_ant_iloc
BTC_BRL_df.head()

In [115]:
## Máximas dos próximos 2 candles
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=-2)
min_2cd_ant_indexer = BTC_BRL_df['Low'].rolling(window=indexer).min()
min_2cd_ant_indexer

Date
2020-10-13         NaN
2020-10-14         NaN
2020-10-15     63505.0
2020-10-16     63650.0
2020-10-17     64418.0
                ...   
2022-04-10    185156.0
2022-04-11    185156.0
2022-04-12    185156.0
2022-04-13    186200.0
2022-04-14    194075.0
Name: Low, Length: 549, dtype: float64

In [ ]:
## Usando o for e iloc

lista_min_2cd_ant = []



In [ ]:
Regra1 =  btc_df['low'] < btc_df['min_2cd_ant']
btc_df['Regra1'] = Regra1
btc_df.head(10)

,open,high,low,close,min_2cd_ant,Regra1
date,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False
2021-01-06,179990.0,197555.0,177210.0,196635.0,150000.0,False
2021-01-07,196752.0,222813.0,194600.0,215874.0,159769.0,False
2021-01-08,216160.0,228045.0,200150.0,222915.0,177210.0,False
2021-01-09,223113.0,225333.0,213593.0,220612.0,194600.0,False


In [ ]:
fig = make_subplots(rows=1, cols=1)

data_range_btc_df = pd.date_range(start='2021-01-01',end='2021-01-04')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False,legend=dict(orientation="h",xanchor="right",x=1,yanchor="bottom",y=1.1),title_text='<b>Regra 1 (low 0 < low 1,2) = True' ,width=1000 ,height=500)
fig.show()

**Regra 2:** Close 0 > Close 1

In [ ]:
## Fechamento do candle anterior
fech_cd_ant = btc_df['close'].rolling(window=1,closed='left').min()
btc_df['fech_cd_ant'] = fech_cd_ant
btc_df.head()

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant
date,,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False,NaN
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False,153480.0
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False,168784.0
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True,173503.0
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False,169820.0


In [ ]:
Regra2 =  btc_df['close'] > btc_df['fech_cd_ant']
btc_df['Regra2'] = Regra2
btc_df.head()

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2
date,,,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False,NaN,False
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False,153480.0,True
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False,168784.0,True
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True,173503.0,False
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False,169820.0,True


In [ ]:
fig = make_subplots(rows=1, cols=1)

data_range_btc_df = pd.date_range(start='2021-01-01',end='2021-01-04')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False,legend=dict(orientation="h",xanchor="right",x=1,yanchor="bottom",y=1.1),title_text=('<b>Regra 2 (close 0 > close 1) = False') ,width=1000 ,height=500)

fig.show()

In [ ]:
## Se Regra 1 e 2 forem verdadeiras → sinal de compra = True
sinal_compra = btc_df['Regra1'] * btc_df['Regra2']
btc_df['sinal_compra'] = sinal_compra
btc_df.head(25)

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra
date,,,,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False,NaN,False,False
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False,153480.0,True,False
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False,168784.0,True,False
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True,173503.0,False,False
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False,169820.0,True,False
2021-01-06,179990.0,197555.0,177210.0,196635.0,150000.0,False,179972.0,True,False
2021-01-07,196752.0,222813.0,194600.0,215874.0,159769.0,False,196635.0,True,False
2021-01-08,216160.0,228045.0,200150.0,222915.0,177210.0,False,215874.0,True,False
2021-01-09,223113.0,225333.0,213593.0,220612.0,194600.0,False,222915.0,False,False


In [ ]:
## Filtro: apenas candles que representaram sinal de compra
sinal_compra_df = btc_df[btc_df.sinal_compra == True]
sinal_compra_df

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra
date,,,,,,,,,
2021-01-22,167340.0,186000.0,155000.0,181268.0,163651.0,True,167696.0,True,True
2021-03-05,277237.0,283451.0,266390.0,279491.0,271140.0,True,277237.0,True,True
2021-03-16,317383.0,322436.0,305262.0,322326.0,313000.0,True,317705.0,True,True
2021-03-23,303155.0,310000.0,296252.0,304195.0,301502.0,True,303155.0,True,True
2021-04-04,332809.0,338597.0,328095.0,337412.0,331500.0,True,332818.0,True,True
2021-05-30,182731.0,192127.0,176727.0,188236.0,178186.0,True,182653.0,True,True
2021-06-20,183146.0,186220.0,172850.0,183349.0,179346.0,True,183116.0,True,True
2021-06-22,160890.0,169364.0,145000.0,163318.0,159301.0,True,160917.0,True,True
2021-06-26,158466.0,163250.0,151151.0,161319.0,156801.0,True,158467.0,True,True


In [ ]:
## Variavel interessante: Qts entradas considerando este ativo, neste intervalo e neste timeframe?
n_sinal_compra_df = len(sinal_compra_df)
n_sinal_compra_df

25

In [ ]:
## Visualização usando um data range
data_range_btc_df = pd.date_range(start='2021-03-01',end='2021-03-31')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra
date,,,,,,,,,
2021-03-01,257538.0,282000.0,256447.0,281400.0,246517.0,False,257551.0,True,False
2021-03-02,281490.0,284121.0,269332.0,277000.0,246517.0,False,281400.0,False,False
2021-03-03,277000.0,299600.0,275353.0,285401.0,256447.0,False,277000.0,True,False
2021-03-04,285401.0,293100.0,271140.0,277237.0,269332.0,False,285401.0,False,False
2021-03-05,277237.0,283451.0,266390.0,279491.0,271140.0,True,277237.0,True,True
2021-03-06,279696.0,282000.0,271285.0,279999.0,266390.0,False,279491.0,True,False
2021-03-07,280000.0,291899.0,279950.0,288621.0,266390.0,False,279999.0,True,False
2021-03-08,289458.0,303500.0,280970.0,303500.0,271285.0,False,288621.0,True,False
2021-03-09,303500.0,319500.0,300665.0,318596.0,279950.0,False,303500.0,True,False


In [ ]:
## Geral 
fig = make_subplots(rows=1, cols=2)

## Figura 1
data_range_btc_df = pd.date_range(start='2021-03-01',end='2021-03-31')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close']), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)",row=1, col=1)
fig.add_vrect(x0='2021-03-02 12:00', x1='2021-03-07 15:00', fillcolor="green", opacity=0.25, line_width=0,row=1, col=1)
fig.add_trace(go.Scatter(x=['2021-03-05'],y=[330000],mode="text",text="<b>Zoom",textposition="top center",showlegend=False),row=1, col=1)

## Figura 2
data_range_btc_df = pd.date_range(start='2021-03-03',end='2021-03-07')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=False), row=1,col=2)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=2)
fig.add_annotation(x='2021-03-05',y=270000, xref="x",yref="y",text="<b>Candle_Sinalizador",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="center",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=2)
fig.add_annotation(x='2021-03-05',y=283452, xref="x",yref="y",text="<b>Compra Max + 1",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="right",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=2)
fig.add_annotation(x='2021-03-07',y=283452, xref="x",yref="y",text="<b>Compra ativada",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=2)
fig.add_trace(go.Scatter(x=['2021-03-04 20:00','2021-03-07 10:00'],y=[283451,283451,283451], mode="lines+text", name="Preço entrada compra",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=2)

## Geral
fig.update_layout(xaxis_rangeslider_visible=False,xaxis2_rangeslider_visible=False,legend=dict(orientation="h",xanchor="right",x=1,yanchor="bottom",y=1.1),title_text='Sinal Compra PFR' ,width=1000 ,height=500)

fig.show()

In [ ]:
## Figura 2 apenas
fig = make_subplots(rows=1, cols=1)

data_range_btc_df = pd.date_range(start='2021-03-03',end='2021-03-07')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)
fig.add_annotation(x='2021-03-05',y=270000, xref="x",yref="y",text="<b>Candle_Sinalizador",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="center",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)
fig.add_annotation(x='2021-03-05',y=283452, xref="x",yref="y",text="<b>Compra Max + 1",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="right",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)
fig.add_annotation(x='2021-03-07',y=283452, xref="x",yref="y",text="<b>Compra ativada",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)
fig.add_trace(go.Scatter(x=['2021-03-04 20:00','2021-03-07 10:00'],y=[283451,283451,283451], mode="lines+text", name="Preço entrada compra",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=1)

## Geral
fig.update_layout(xaxis_rangeslider_visible=False,legend=dict(orientation="h",xanchor="right",x=1,yanchor="bottom",y=1.1),title_text='Sinal Compra PFR' ,width=1000 ,height=500)

fig.show()

In [ ]:
## O que aconteceu depois?

fig = make_subplots(rows=1, cols=1)

data_range_btc_df = pd.date_range(start='2021-03-03',end='2021-03-12')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)
fig.add_annotation(x='2021-03-05',y=270000, xref="x",yref="y",text="<b>Candle_Sinalizador",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="center",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)
fig.add_annotation(x='2021-03-05',y=283452, xref="x",yref="y",text="<b>Compra Max + 1",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="right",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)
fig.add_annotation(x='2021-03-07',y=283452, xref="x",yref="y",text="<b>Compra ativada",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)
fig.add_trace(go.Scatter(x=['2021-03-04 20:00','2021-03-07 10:00'],y=[283451,283451,283451], mode="lines+text", name="Preço entrada compra",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=1)

## Geral
fig.update_layout(xaxis_rangeslider_visible=False,legend=dict(orientation="h",xanchor="right",x=1,yanchor="bottom",y=1.1),title_text='Sinal Compra PFR' ,width=1000 ,height=500)

fig.show()

## **3.3. Entradas de COMPRA executadas**

Se Preco > High (candle sinalizador) + Min_Incr = Compra executada<p>
PS: Definiremos que o período máx para acontecer são 2 candles

In [ ]:
## Criar uma coluna para informar se entrei ou nao na operação. Daremos uma janela de 2 candles no máx para entrar.
## Para células posteriores, usar o FixedForwardWindowIndexer

## Máximas dos próximos 2 candles
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
max_cd_sig_prox = btc_df['high'].rolling(window=indexer).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=3)
max_cd_sig_prox2 = btc_df['high'].rolling(window=indexer).max()
btc_df['max_cd_sig_prox'] = max_cd_sig_prox
btc_df['max_cd_sig_prox2'] = max_cd_sig_prox2
btc_df.head()

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra,max_cd_sig_prox,max_cd_sig_prox2
date,,,,,,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False,NaN,False,False,173900.0,180925.0
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False,153480.0,True,False,180925.0,180925.0
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False,168784.0,True,False,180925.0,182966.0
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True,173503.0,False,False,182966.0,197555.0
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False,169820.0,True,False,197555.0,222813.0


In [ ]:
## Entrada compra ativada? Lembrando que tick_min foi definido em params
btc_df['Compra_ativ_1cd'] = (btc_df['max_cd_sig_prox'] > (btc_df['high']+1*tick_min)) *  btc_df['sinal_compra']
btc_df['Compra_ativ_2cd'] = (btc_df['max_cd_sig_prox2'] > (btc_df['high']+1*tick_min)) *  btc_df['sinal_compra']
btc_df.head(25)

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra,max_cd_sig_prox,max_cd_sig_prox2,Compra_ativ_1cd,Compra_ativ_2cd
date,,,,,,,,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False,NaN,False,False,173900.0,180925.0,False,False
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False,153480.0,True,False,180925.0,180925.0,False,False
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False,168784.0,True,False,180925.0,182966.0,False,False
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True,173503.0,False,False,182966.0,197555.0,False,False
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False,169820.0,True,False,197555.0,222813.0,False,False
2021-01-06,179990.0,197555.0,177210.0,196635.0,150000.0,False,179972.0,True,False,222813.0,228045.0,False,False
2021-01-07,196752.0,222813.0,194600.0,215874.0,159769.0,False,196635.0,True,False,228045.0,228045.0,False,False
2021-01-08,216160.0,228045.0,200150.0,222915.0,177210.0,False,215874.0,True,False,228045.0,228045.0,False,False
2021-01-09,223113.0,225333.0,213593.0,220612.0,194600.0,False,222915.0,False,False,225333.0,225333.0,False,False


In [ ]:
data_range_btc_df = pd.date_range(start='2021-01-19',end='2021-01-24')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.add_annotation(x='2021-01-22',y=160000, xref="x",yref="y",text="<b>Candle_Sinalizador",showarrow=True,font=dict(family="Courier New, monospace",size=16,color="black"),align="center",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black")
fig.add_annotation(x='2021-01-22',y=186001, xref="x",yref="y",text="<b>Compra Max + 1",showarrow=True,font=dict(family="Courier New, monospace",size=16,color="black"),align="right",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black")
fig.add_trace(go.Scatter(x=['2021-01-21 20:00','2021-01-24 10:00'],y=[186001,186001,186001], mode="lines+text", name="Preço entrada compra",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=1)

fig.add_annotation(x='2021-01-24',y=186001, xref="x",yref="y",text="<b>Compra NÃO ativada",showarrow=True,font=dict(family="Courier New, monospace",size=12,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Sinal Compra PFR',width=800 ,height=800)
fig.show()

In [ ]:
data_range_btc_df = pd.date_range(start='2021-01-19',end='2021-01-31')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.add_annotation(x='2021-01-22',y=160000, xref="x",yref="y",text="<b>Candle_Sinalizador",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="center",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black")
fig.add_annotation(x='2021-01-22',y=186001, xref="x",yref="y",text="<b>Compra Max + 1",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="right",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black")
fig.add_trace(go.Scatter(x=['2021-01-21 20:00','2021-01-24 10:00'],y=[186001,186001,186001], mode="lines+text", name="Preço entrada compra",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=1)

fig.add_annotation(x='2021-01-24',y=186001, xref="x",yref="y",text="<b>Compra NÃO ativada",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Sinal Compra PFR',width=800 ,height=800)
fig.show()

In [ ]:
## Filtro: apenas candles que representaram sinal de compra (apresentado acima mas agora com entrada ativada)
sinal_compra_df = btc_df[btc_df.sinal_compra == True]
sinal_compra_df

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra,max_cd_sig_prox,max_cd_sig_prox2,Compra_ativ_1cd,Compra_ativ_2cd
date,,,,,,,,,,,,,
2021-01-22,167340.0,186000.0,155000.0,181268.0,163651.0,True,167696.0,True,True,186000.0,186000.0,False,False
2021-03-05,277237.0,283451.0,266390.0,279491.0,271140.0,True,277237.0,True,True,283451.0,291899.0,False,True
2021-03-16,317383.0,322436.0,305262.0,322326.0,313000.0,True,317705.0,True,True,331244.0,335000.0,True,True
2021-03-23,303155.0,310000.0,296252.0,304195.0,301502.0,True,303155.0,True,True,318778.0,318778.0,True,True
2021-04-04,332809.0,338597.0,328095.0,337412.0,331500.0,True,332818.0,True,True,339093.0,339654.0,True,True
2021-05-30,182731.0,192127.0,176727.0,188236.0,178186.0,True,182653.0,True,True,196776.0,198465.0,True,True
2021-06-20,183146.0,186220.0,172850.0,183349.0,179346.0,True,183116.0,True,True,186220.0,186220.0,False,False
2021-06-22,160890.0,169364.0,145000.0,163318.0,159301.0,True,160917.0,True,True,177000.0,177000.0,True,True
2021-06-26,158466.0,163250.0,151151.0,161319.0,156801.0,True,158467.0,True,True,172443.0,174804.0,True,True


In [ ]:
## Filtro: apenas candles que representaram sinal de compra (apresentado acima mas agora com entrada ativada)
compras_ativadas_1cd_df = sinal_compra_df[sinal_compra_df.Compra_ativ_1cd == True]
compras_ativadas_2cd_df = sinal_compra_df[sinal_compra_df.Compra_ativ_2cd == True]
print(len(compras_ativadas_1cd_df))
len(compras_ativadas_2cd_df)

20


23

## **3.4. Definindo os alvos gain x stop loss**

candle sinalizador = cdsig<p>
Alvo_Fibonacci = 1.61

*Stop loss*:<p>
Preco < Low_cdsig - Min_Incr -> Sell to Cover Stop<p>

*Alvos Gain* (2):<p>
Gain1 = Se Preco > (Max_cdsig + Min_Incr) + Amplitude_cdsig = Sell to Cover Limit<p>
Gain2 = Se Preco > (Max_cdsig + Min_Incr) + (Amplitude_cdsig * 1.61) = Sell to Cover Limit

In [ ]:
## Amplitude (max - min) do candle sinalizador
btc_df['range'] = btc_df['high']-btc_df['low']

## Preço do Stop loss: Min cd sinal - tick_min

btc_df['stop_loss'] = btc_df['low'] - tick_min

## Preco Alvo Gain: Max cd sinal + 1 + Range candle sinal

btc_df['alvo_gain1'] = btc_df['high'] + tick_min + btc_df['range']
btc_df['alvo_gain2'] = btc_df['high'] + tick_min + (Alvo_Fibonacci * btc_df['range'])

btc_df

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra,max_cd_sig_prox,max_cd_sig_prox2,Compra_ativ_1cd,Compra_ativ_2cd,range,stop_loss,alvo_gain1,alvo_gain2
date,,,,,,,,,,,,,,,,,
2021-01-01,152056.0,154167.0,150911.0,153480.0,NaN,False,NaN,False,False,173900.0,180925.0,False,False,3256.0,150910.0,157424.0,159410.16
2021-01-02,153478.0,173900.0,152171.0,168784.0,NaN,False,153480.0,True,False,180925.0,180925.0,False,False,21729.0,152170.0,195630.0,208884.69
2021-01-03,168885.0,180925.0,167897.0,173503.0,150911.0,False,168784.0,True,False,180925.0,182966.0,False,False,13028.0,167896.0,193954.0,201901.08
2021-01-04,173504.0,175981.0,150000.0,169820.0,152171.0,True,173503.0,False,False,182966.0,197555.0,False,False,25981.0,149999.0,201963.0,217811.41
2021-01-05,169825.0,182966.0,159769.0,179972.0,150000.0,False,169820.0,True,False,197555.0,222813.0,False,False,23197.0,159768.0,206164.0,220314.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,290189.0,295785.0,287534.0,288378.0,283013.0,False,290188.0,False,False,295785.0,295785.0,False,False,8251.0,287533.0,304037.0,309070.11
2021-12-28,288339.0,288339.0,269171.0,270226.0,283013.0,True,288378.0,False,False,288339.0,288339.0,False,False,19168.0,269170.0,307508.0,319200.48
2021-12-29,270357.0,274941.0,264773.0,266975.0,269171.0,True,270226.0,False,False,280000.0,280000.0,False,False,10168.0,264772.0,285110.0,291312.48


## **3.5. Verificar onde o preço chegou primeiro: stop loss ou gain?**

In [ ]:
## Primeiro devo calcular a min e max dos 'próximos candles' (lembrando que inclui o candle sinalizador)
## Entretanto, dependendo do tempo da operação pode ficar chato fazer na mão (além de grande risco de erros)

indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
min_cd_prox1 = btc_df['low'].rolling(window=indexer).min()
max_cd_prox1 = btc_df['high'].rolling(window=indexer).max()

indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=3)
min_cd_prox2 = btc_df['low'].rolling(window=indexer).min()
max_cd_prox2 = btc_df['high'].rolling(window=indexer).max()

<img src="https://s3.manualzz.com/store/data/035682191_1-cb12413719b59b4f0190c1355bbe02ad-360x466.png" width="300">


In [ ]:
## Aqui você pode determinar até quantos candles ficará na operação. Aqui depende mt do setup. Alguns encerram por tempo, outros so qd chegar nos alvos...
## Copiar o outputr e colar na linha de código abaixo

for i in range(1,11):
  print('indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=',i+1,')',sep='')
  print('btc_df["min_cd_prox',i,'"] = btc_df["low"].rolling(window=',i+1,').min()',sep='')
  print('btc_df["max_cd_prox',i,'"] = btc_df["high"].rolling(window=',i+1,').max()',sep='')

indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
btc_df["min_cd_prox1"] = btc_df["low"].rolling(window=2).min()
btc_df["max_cd_prox1"] = btc_df["high"].rolling(window=2).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=3)
btc_df["min_cd_prox2"] = btc_df["low"].rolling(window=3).min()
btc_df["max_cd_prox2"] = btc_df["high"].rolling(window=3).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=4)
btc_df["min_cd_prox3"] = btc_df["low"].rolling(window=4).min()
btc_df["max_cd_prox3"] = btc_df["high"].rolling(window=4).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=5)
btc_df["min_cd_prox4"] = btc_df["low"].rolling(window=5).min()
btc_df["max_cd_prox4"] = btc_df["high"].rolling(window=5).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=6)
btc_df["min_cd_prox5"] = btc_df["low"].rolling(window=6).min()
btc_df["max_cd_prox5"] = btc_df["high"].rolling(window=6).max()
indexer = pd.api.indexers.Fixe

In [ ]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
btc_df["min_cd_prox1"] = btc_df["low"].rolling(window=2).min()
btc_df["max_cd_prox1"] = btc_df["high"].rolling(window=2).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=3)
btc_df["min_cd_prox2"] = btc_df["low"].rolling(window=3).min()
btc_df["max_cd_prox2"] = btc_df["high"].rolling(window=3).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=4)
btc_df["min_cd_prox3"] = btc_df["low"].rolling(window=4).min()
btc_df["max_cd_prox3"] = btc_df["high"].rolling(window=4).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=5)
btc_df["min_cd_prox4"] = btc_df["low"].rolling(window=5).min()
btc_df["max_cd_prox4"] = btc_df["high"].rolling(window=5).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=6)
btc_df["min_cd_prox5"] = btc_df["low"].rolling(window=6).min()
btc_df["max_cd_prox5"] = btc_df["high"].rolling(window=6).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=7)
btc_df["min_cd_prox6"] = btc_df["low"].rolling(window=7).min()
btc_df["max_cd_prox6"] = btc_df["high"].rolling(window=7).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=8)
btc_df["min_cd_prox7"] = btc_df["low"].rolling(window=8).min()
btc_df["max_cd_prox7"] = btc_df["high"].rolling(window=8).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=9)
btc_df["min_cd_prox8"] = btc_df["low"].rolling(window=9).min()
btc_df["max_cd_prox8"] = btc_df["high"].rolling(window=9).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=10)
btc_df["min_cd_prox9"] = btc_df["low"].rolling(window=10).min()
btc_df["max_cd_prox9"] = btc_df["high"].rolling(window=10).max()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=11)
btc_df["min_cd_prox10"] = btc_df["low"].rolling(window=11).min()
btc_df["max_cd_prox10"] = btc_df["high"].rolling(window=11).max()

In [ ]:
print(len(btc_df.columns))
btc_df.columns

37


Index(['open', 'high', 'low', 'close', 'min_2cd_ant', 'Regra1', 'fech_cd_ant',
       'Regra2', 'sinal_compra', 'max_cd_sig_prox', 'max_cd_sig_prox2',
       'Compra_ativ_1cd', 'Compra_ativ_2cd', 'range', 'stop_loss',
       'alvo_gain1', 'alvo_gain2', 'min_cd_prox1', 'max_cd_prox1',
       'min_cd_prox2', 'max_cd_prox2', 'min_cd_prox3', 'max_cd_prox3',
       'min_cd_prox4', 'max_cd_prox4', 'min_cd_prox5', 'max_cd_prox5',
       'min_cd_prox6', 'max_cd_prox6', 'min_cd_prox7', 'max_cd_prox7',
       'min_cd_prox8', 'max_cd_prox8', 'min_cd_prox9', 'max_cd_prox9',
       'min_cd_prox10', 'max_cd_prox10'],
      dtype='object')

In [ ]:
print(btc_df.columns.get_loc("stop_loss"))
print(btc_df.columns.get_loc("alvo_gain1"))
print(btc_df.columns.get_loc("alvo_gain2"))
print(btc_df.columns.get_loc("min_cd_prox1"))
print(btc_df.columns.get_loc("max_cd_prox1"))
print(btc_df.columns.get_loc("min_cd_prox10"))
print(btc_df.columns.get_loc("max_cd_prox10"))

14
15
16
17
18
35
36


In [ ]:
btc_df[btc_df.columns[12]]

date
2021-01-01    False
2021-01-02    False
2021-01-03    False
2021-01-04    False
2021-01-05    False
              ...  
2021-12-27    False
2021-12-28    False
2021-12-29    False
2021-12-30    False
2021-12-31    False
Name: Compra_ativ_2cd, Length: 365, dtype: bool

Muitas formas potenciais de calcular: pivot table, if, while, rolling (axis =1)...<p>
Vamos usar esse mais simples como feito no anterior

In [ ]:
for i in range(1,11):
  print('btc_df["Loss_',i,'"] = btc_df["min_cd_prox',i,'"] < btc_df["stop_loss"]',sep='')
  print('btc_df["Gain1_',i,'"] = btc_df["max_cd_prox',i,'"] > btc_df["alvo_gain1"]',sep='')
  print('btc_df["Gain2_',i,'"] = btc_df["max_cd_prox',i,'"] > btc_df["alvo_gain2"]',sep='')

btc_df["Loss_1"] = btc_df["min_cd_prox1"] < btc_df["stop_loss"]
btc_df["Gain1_1"] = btc_df["max_cd_prox1"] > btc_df["alvo_gain1"]
btc_df["Gain2_1"] = btc_df["max_cd_prox1"] > btc_df["alvo_gain2"]
btc_df["Loss_2"] = btc_df["min_cd_prox2"] < btc_df["stop_loss"]
btc_df["Gain1_2"] = btc_df["max_cd_prox2"] > btc_df["alvo_gain1"]
btc_df["Gain2_2"] = btc_df["max_cd_prox2"] > btc_df["alvo_gain2"]
btc_df["Loss_3"] = btc_df["min_cd_prox3"] < btc_df["stop_loss"]
btc_df["Gain1_3"] = btc_df["max_cd_prox3"] > btc_df["alvo_gain1"]
btc_df["Gain2_3"] = btc_df["max_cd_prox3"] > btc_df["alvo_gain2"]
btc_df["Loss_4"] = btc_df["min_cd_prox4"] < btc_df["stop_loss"]
btc_df["Gain1_4"] = btc_df["max_cd_prox4"] > btc_df["alvo_gain1"]
btc_df["Gain2_4"] = btc_df["max_cd_prox4"] > btc_df["alvo_gain2"]
btc_df["Loss_5"] = btc_df["min_cd_prox5"] < btc_df["stop_loss"]
btc_df["Gain1_5"] = btc_df["max_cd_prox5"] > btc_df["alvo_gain1"]
btc_df["Gain2_5"] = btc_df["max_cd_prox5"] > btc_df["alvo_gain2"]
btc_df["Loss_6"] = b

In [ ]:
btc_df["Loss_1"] = btc_df["min_cd_prox1"] < btc_df["stop_loss"]
btc_df["Gain1_1"] = btc_df["max_cd_prox1"] > btc_df["alvo_gain1"]
btc_df["Gain2_1"] = btc_df["max_cd_prox1"] > btc_df["alvo_gain2"]
btc_df["Loss_2"] = btc_df["min_cd_prox2"] < btc_df["stop_loss"]
btc_df["Gain1_2"] = btc_df["max_cd_prox2"] > btc_df["alvo_gain1"]
btc_df["Gain2_2"] = btc_df["max_cd_prox2"] > btc_df["alvo_gain2"]
btc_df["Loss_3"] = btc_df["min_cd_prox3"] < btc_df["stop_loss"]
btc_df["Gain1_3"] = btc_df["max_cd_prox3"] > btc_df["alvo_gain1"]
btc_df["Gain2_3"] = btc_df["max_cd_prox3"] > btc_df["alvo_gain2"]
btc_df["Loss_4"] = btc_df["min_cd_prox4"] < btc_df["stop_loss"]
btc_df["Gain1_4"] = btc_df["max_cd_prox4"] > btc_df["alvo_gain1"]
btc_df["Gain2_4"] = btc_df["max_cd_prox4"] > btc_df["alvo_gain2"]
btc_df["Loss_5"] = btc_df["min_cd_prox5"] < btc_df["stop_loss"]
btc_df["Gain1_5"] = btc_df["max_cd_prox5"] > btc_df["alvo_gain1"]
btc_df["Gain2_5"] = btc_df["max_cd_prox5"] > btc_df["alvo_gain2"]
btc_df["Loss_6"] = btc_df["min_cd_prox6"] < btc_df["stop_loss"]
btc_df["Gain1_6"] = btc_df["max_cd_prox6"] > btc_df["alvo_gain1"]
btc_df["Gain2_6"] = btc_df["max_cd_prox6"] > btc_df["alvo_gain2"]
btc_df["Loss_7"] = btc_df["min_cd_prox7"] < btc_df["stop_loss"]
btc_df["Gain1_7"] = btc_df["max_cd_prox7"] > btc_df["alvo_gain1"]
btc_df["Gain2_7"] = btc_df["max_cd_prox7"] > btc_df["alvo_gain2"]
btc_df["Loss_8"] = btc_df["min_cd_prox8"] < btc_df["stop_loss"]
btc_df["Gain1_8"] = btc_df["max_cd_prox8"] > btc_df["alvo_gain1"]
btc_df["Gain2_8"] = btc_df["max_cd_prox8"] > btc_df["alvo_gain2"]
btc_df["Loss_9"] = btc_df["min_cd_prox9"] < btc_df["stop_loss"]
btc_df["Gain1_9"] = btc_df["max_cd_prox9"] > btc_df["alvo_gain1"]
btc_df["Gain2_9"] = btc_df["max_cd_prox9"] > btc_df["alvo_gain2"]
btc_df["Loss_10"] = btc_df["min_cd_prox10"] < btc_df["stop_loss"]
btc_df["Gain1_10"] = btc_df["max_cd_prox10"] > btc_df["alvo_gain1"]
btc_df["Gain2_10"] = btc_df["max_cd_prox10"] > btc_df["alvo_gain2"]

In [ ]:
## Filtrar apenas os com entrada de compra ativada
compras_ativadas_1cd_df = btc_df[btc_df.Compra_ativ_1cd == True]
compras_ativadas_2cd_df = btc_df[btc_df.Compra_ativ_2cd == True]
compras_ativadas_2cd_df

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra,max_cd_sig_prox,max_cd_sig_prox2,Compra_ativ_1cd,Compra_ativ_2cd,range,stop_loss,alvo_gain1,alvo_gain2,min_cd_prox1,max_cd_prox1,min_cd_prox2,max_cd_prox2,min_cd_prox3,max_cd_prox3,min_cd_prox4,max_cd_prox4,min_cd_prox5,max_cd_prox5,min_cd_prox6,max_cd_prox6,min_cd_prox7,max_cd_prox7,min_cd_prox8,max_cd_prox8,min_cd_prox9,max_cd_prox9,min_cd_prox10,max_cd_prox10,Loss_1,Gain1_1,Gain2_1,Loss_2,Gain1_2,Gain2_2,Loss_3,Gain1_3,Gain2_3,Loss_4,Gain1_4,Gain2_4,Loss_5,Gain1_5,Gain2_5,Loss_6,Gain1_6,Gain2_6,Loss_7,Gain1_7,Gain2_7,Loss_8,Gain1_8,Gain2_8,Loss_9,Gain1_9,Gain2_9,Loss_10,Gain1_10,Gain2_10
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-05,277237.0,283451.0,266390.0,279491.0,271140.0,True,277237.0,True,True,283451.0,291899.0,False,True,17061.0,266389.0,300513.0,310920.21,266390.0,293100.0,266390.0,299600.0,266390.0,299600.0,256447.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,302000.0,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False
2021-03-16,317383.0,322436.0,305262.0,322326.0,313000.0,True,317705.0,True,True,331244.0,335000.0,True,True,17174.0,305261.0,339611.0,350087.14,305262.0,344150.0,305262.0,345300.0,305262.0,346263.0,305262.0,346263.0,305262.0,346263.0,305262.0,346263.0,300665.0,346263.0,280970.0,346263.0,279950.0,346263.0,271285.0,346263.0,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,False,True,True,False,True,True,False,True,True,False
2021-03-23,303155.0,310000.0,296252.0,304195.0,301502.0,True,303155.0,True,True,318778.0,318778.0,True,True,13748.0,296251.0,323749.0,332135.28,296252.0,325549.0,296252.0,326996.0,296252.0,331000.0,296252.0,331000.0,296252.0,335000.0,296252.0,335000.0,296252.0,335000.0,296252.0,344150.0,296252.0,345300.0,296252.0,346263.0,False,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False,True,True,False,True,True,False,True,True,False,True,True,False,True,True
2021-04-04,332809.0,338597.0,328095.0,337412.0,331500.0,True,332818.0,True,True,339093.0,339654.0,True,True,10502.0,328094.0,349100.0,355506.22,328095.0,342116.0,328095.0,344298.0,328095.0,344298.0,328095.0,344298.0,328095.0,344298.0,316296.0,344298.0,316009.0,344298.0,312901.0,344298.0,293200.0,344298.0,288502.0,344298.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False
2021-05-30,182731.0,192127.0,176727.0,188236.0,178186.0,True,182653.0,True,True,196776.0,198465.0,True,True,15400.0,176726.0,207528.0,216922.00,176727.0,196773.0,176727.0,205598.0,176727.0,215500.0,176727.0,217617.0,176727.0,217617.0,176727.0,217617.0,173177.0,217617.0,173177.0,217617.0,173177.0,226213.0,173177.0,231400.0,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2021-06-22,160890.0,169364.0,145000.0,163318.0,159301.0,True,160917.0,True,True,177000.0,177000.0,True,True,24364.0,144999.0,193729.0,208591.04,145000.0,184000.0,145000.0,186220.0,145000.0,187461.0,145000.0,193197.0,145000.0,201595.0,145000.0,205794.0,145000.0,209898.0,145000.0,210000.0,145000.0,210000.0,145000.0,210000.0,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,True,True,False,True,True,False,True,True
2021-06-26,158466.0,163250.0,151151.0,161319.0,156801.0,True,158467.0,True,True,172443.0,174804.0,True,True,12099.0,151150.0,175350.0,182730.39,151151.0,175799.0,151151.0,175799.0,151151.0,177000.0,145000.0,177000.0,145000.0,184000.0,145000.0,186220.0,145000.0,187461.0,145000.0,193197.0,145000.0,201595.0

In [ ]:
compras_ativadas_2cd_df.columns

Index(['open', 'high', 'low', 'close', 'min_2cd_ant', 'Regra1', 'fech_cd_ant',
       'Regra2', 'sinal_compra', 'max_cd_sig_prox', 'max_cd_sig_prox2',
       'Compra_ativ_1cd', 'Compra_ativ_2cd', 'range', 'stop_loss',
       'alvo_gain1', 'alvo_gain2', 'min_cd_prox1', 'max_cd_prox1',
       'min_cd_prox2', 'max_cd_prox2', 'min_cd_prox3', 'max_cd_prox3',
       'min_cd_prox4', 'max_cd_prox4', 'min_cd_prox5', 'max_cd_prox5',
       'min_cd_prox6', 'max_cd_prox6', 'min_cd_prox7', 'max_cd_prox7',
       'min_cd_prox8', 'max_cd_prox8', 'min_cd_prox9', 'max_cd_prox9',
       'min_cd_prox10', 'max_cd_prox10', 'Loss_1', 'Gain1_1', 'Gain2_1',
       'Loss_2', 'Gain1_2', 'Gain2_2', 'Loss_3', 'Gain1_3', 'Gain2_3',
       'Loss_4', 'Gain1_4', 'Gain2_4', 'Loss_5', 'Gain1_5', 'Gain2_5',
       'Loss_6', 'Gain1_6', 'Gain2_6', 'Loss_7', 'Gain1_7', 'Gain2_7',
       'Loss_8', 'Gain1_8', 'Gain2_8', 'Loss_9', 'Gain1_9', 'Gain2_9',
       'Loss_10', 'Gain1_10', 'Gain2_10'],
      dtype='object')

In [ ]:
print(compras_ativadas_2cd_df.columns.get_loc("Loss_1"))
print(compras_ativadas_2cd_df.columns.get_loc("Gain2_10"))

37
66


In [ ]:
## Mt complexo ainda
compras_ativadas_2cd_df[compras_ativadas_2cd_df.columns[37:67]]

,Loss_1,Gain1_1,Gain2_1,Loss_2,Gain1_2,Gain2_2,Loss_3,Gain1_3,Gain2_3,Loss_4,Gain1_4,Gain2_4,Loss_5,Gain1_5,Gain2_5,Loss_6,Gain1_6,Gain2_6,Loss_7,Gain1_7,Gain2_7,Loss_8,Gain1_8,Gain2_8,Loss_9,Gain1_9,Gain2_9,Loss_10,Gain1_10,Gain2_10
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-05,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False
2021-03-16,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,False,True,True,False,True,True,False,True,True,False
2021-03-23,False,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False,True,True,False,True,True,False,True,True,False,True,True,False,True,True
2021-04-04,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False
2021-05-30,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2021-06-22,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,True,True,False,True,True,False,True,True
2021-06-26,False,True,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2021-07-30,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False
2021-08-04,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False


In [ ]:
compras_ativadas_2cd_df.head()

,open,high,low,close,min_2cd_ant,Regra1,fech_cd_ant,Regra2,sinal_compra,max_cd_sig_prox,max_cd_sig_prox2,Compra_ativ_1cd,Compra_ativ_2cd,range,stop_loss,alvo_gain1,alvo_gain2,min_cd_prox1,max_cd_prox1,min_cd_prox2,max_cd_prox2,min_cd_prox3,max_cd_prox3,min_cd_prox4,max_cd_prox4,min_cd_prox5,max_cd_prox5,min_cd_prox6,max_cd_prox6,min_cd_prox7,max_cd_prox7,min_cd_prox8,max_cd_prox8,min_cd_prox9,max_cd_prox9,min_cd_prox10,max_cd_prox10,Loss_1,Gain1_1,Gain2_1,Loss_2,Gain1_2,Gain2_2,Loss_3,Gain1_3,Gain2_3,Loss_4,Gain1_4,Gain2_4,Loss_5,Gain1_5,Gain2_5,Loss_6,Gain1_6,Gain2_6,Loss_7,Gain1_7,Gain2_7,Loss_8,Gain1_8,Gain2_8,Loss_9,Gain1_9,Gain2_9,Loss_10,Gain1_10,Gain2_10
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-05,277237.0,283451.0,266390.0,279491.0,271140.0,True,277237.0,True,True,283451.0,291899.0,False,True,17061.0,266389.0,300513.0,310920.21,266390.0,293100.0,266390.0,299600.0,266390.0,299600.0,256447.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,299600.0,246517.0,302000.0,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False
2021-03-16,317383.0,322436.0,305262.0,322326.0,313000.0,True,317705.0,True,True,331244.0,335000.0,True,True,17174.0,305261.0,339611.0,350087.14,305262.0,344150.0,305262.0,345300.0,305262.0,346263.0,305262.0,346263.0,305262.0,346263.0,305262.0,346263.0,300665.0,346263.0,280970.0,346263.0,279950.0,346263.0,271285.0,346263.0,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,False,True,True,False,True,True,False,True,True,False
2021-03-23,303155.0,310000.0,296252.0,304195.0,301502.0,True,303155.0,True,True,318778.0,318778.0,True,True,13748.0,296251.0,323749.0,332135.28,296252.0,325549.0,296252.0,326996.0,296252.0,331000.0,296252.0,331000.0,296252.0,335000.0,296252.0,335000.0,296252.0,335000.0,296252.0,344150.0,296252.0,345300.0,296252.0,346263.0,False,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False,True,True,False,True,True,False,True,True,False,True,True,False,True,True
2021-04-04,332809.0,338597.0,328095.0,337412.0,331500.0,True,332818.0,True,True,339093.0,339654.0,True,True,10502.0,328094.0,349100.0,355506.22,328095.0,342116.0,328095.0,344298.0,328095.0,344298.0,328095.0,344298.0,328095.0,344298.0,316296.0,344298.0,316009.0,344298.0,312901.0,344298.0,293200.0,344298.0,288502.0,344298.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False
2021-05-30,182731.0,192127.0,176727.0,188236.0,178186.0,True,182653.0,True,True,196776.0,198465.0,True,True,15400.0,176726.0,207528.0,216922.00,176727.0,196773.0,176727.0,205598.0,176727.0,215500.0,176727.0,217617.0,176727.0,217617.0,176727.0,217617.0,173177.0,217617.0,173177.0,217617.0,173177.0,226213.0,173177.0,231400.0,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [ ]:
data_range_btc_df = pd.date_range(start='2021-03-01',end='2021-03-10')
btc_df_index_range = btc_df[btc_df.index.isin(data_range_btc_df)]
btc_df_index_range

fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_df_index_range.index, open=btc_df_index_range['open'], high = btc_df_index_range['high'], low=btc_df_index_range['low'], close=btc_df_index_range['close'],showlegend=True), row=1,col=1)
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.add_annotation(x='2021-03-05',y=275000, xref="x",yref="y",text="<b>Candle_Sinalizador",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="center",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black")
fig.add_annotation(x='2021-03-05',y=283452, xref="x",yref="y",text="<b>Compra Max + 1",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="right",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black")
fig.add_trace(go.Scatter(x=['2021-03-05 00:00','2021-03-07 10:00'],y=[283452,283452,283452], mode="lines+text", name="Preço entrada compra",
                         line = dict(color='purple', width=2, dash='dash')),row=1, col=1)
fig.add_annotation(x='2021-03-07',y=283452, xref="x",yref="y",text="<b>Compra ativada",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)

fig.add_trace(go.Scatter(x=['2021-03-05 00:00','2021-03-07 10:00'],y=[266390,266390,266390], mode="lines+text", name="Stop Loss",
                         line = dict(color='red', width=2, dash='dash')),row=1, col=1)
fig.add_annotation(x='2021-03-05',y=266390, xref="x",yref="y",text="<b>Stop Loss",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)

## Alvos Gain

fig.add_trace(go.Scatter(x=['2021-03-05 00:00','2021-03-10 10:00'],y=[300513,300513,300513], mode="lines+text", name="Alvo Gain (Range)",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=1)
fig.add_annotation(x='2021-03-05',y=300513, xref="x",yref="y",text="<b>Alvo Gain (Range)",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)

fig.add_trace(go.Scatter(x=['2021-03-05 00:00','2021-03-10 10:00'],y=[310920,310920,310920], mode="lines+text", name="Alvo Gain (Fibo)",
                         line = dict(color='green', width=2, dash='dash')),row=1, col=1)
fig.add_annotation(x='2021-03-05',y=310920, xref="x",yref="y",text="<b>Alvo Gain (Fibo)",showarrow=True,font=dict(family="Courier New, monospace",size=10,color="black"),align="left",arrowhead=2,
                   arrowsize=1,arrowwidth=2,arrowcolor="black",row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Sinal Compra PFR',width=1000 ,height=700)
fig.show()

In [ ]:
compras_ativadas_2cd_df_long = pd.melt(compras_ativadas_2cd_df,ignore_index=False)
compras_ativadas_2cd_df_long

,variable,value
date,,
2021-03-05,open,277237.0
2021-03-16,open,317383.0
2021-03-23,open,303155.0
2021-04-04,open,332809.0
2021-05-30,open,182731.0
...,...,...
2021-11-19,Gain2_10,True
2021-11-23,Gain2_10,True
2021-12-11,Gain2_10,True


In [ ]:
Loss01 = compras_ativadas_2cd_df_long[(compras_ativadas_2cd_df_long.variable == 'Loss_1') & (compras_ativadas_2cd_df_long.value == True)]
Loss01

,variable,value
date,,


In [ ]:
Gain1_1 = compras_ativadas_2cd_df_long[(compras_ativadas_2cd_df_long.variable == 'Gain1_1') & (compras_ativadas_2cd_df_long.value == True)]
Gain1_1

,variable,value
date,,
2021-03-16,Gain1_1,True
2021-03-23,Gain1_1,True
2021-06-26,Gain1_1,True
2021-08-25,Gain1_1,True
2021-10-23,Gain1_1,True


In [ ]:
Gain2_1 = compras_ativadas_2cd_df_long[(compras_ativadas_2cd_df_long.variable == 'Gain2_1') & (compras_ativadas_2cd_df_long.value == True)]
Gain2_1

,variable,value
date,,


In [ ]:
 = compras_ativadas_2cd_df_long[(compras_ativadas_2cd_df_long.value == True] & compras_ativadas_2cd_df.columns.get_loc()
print(compras_ativadas_2cd_df.columns.get_loc("Loss_1"))

,variable,value
date,,
2021-03-05,Loss_10,True
2021-03-16,Loss_10,True
2021-04-04,Loss_10,True
2021-05-30,Loss_10,True
2021-06-26,Loss_10,True
2021-07-30,Loss_10,True
2021-08-04,Loss_10,True
2021-08-05,Loss_10,True
2021-08-18,Loss_10,True


In [ ]:
compras_ativadas_2cd_df['Detector'] = (1==1)
compras_ativadas_2cd_df['Detector']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



date
2021-03-05    True
2021-03-16    True
2021-03-23    True
2021-04-04    True
2021-05-30    True
2021-06-22    True
2021-06-26    True
2021-07-30    True
2021-08-04    True
2021-08-05    True
2021-08-18    True
2021-08-25    True
2021-09-01    True
2021-09-29    True
2021-10-23    True
2021-10-28    True
2021-11-06    True
2021-11-17    True
2021-11-19    True
2021-11-23    True
2021-12-11    True
2021-12-18    True
2021-12-26    True
Name: Detector, dtype: bool

In [ ]:
lista_resultados = []

preco_atingiu = 'nada'

for i in range(37,len(compras_ativadas_2cd_df.columns)):
  if compras_ativadas_2cd_df[compras_ativadas_2cd_df.columns[i]] == compras_ativadas_2cd_df[compras_ativadas_2cd_df.columns['Detector']]:
    preco_atingiu = compras_ativadas_2cd_df.columns[i]
    lista_resultados.append()
  else:
    pass

IndexError: ignored

In [ ]:
compras_ativadas_2cd_df.columns[37]

'Loss_1'

In [ ]:
compras_ativadas_2cd_df[compras_ativadas_2cd_df.columns[37]] 

date
2021-03-05    False
2021-03-16    False
2021-03-23    False
2021-04-04    False
2021-05-30    False
2021-06-22    False
2021-06-26    False
2021-07-30    False
2021-08-04    False
2021-08-05    False
2021-08-18    False
2021-08-25    False
2021-09-01    False
2021-09-29    False
2021-10-23    False
2021-10-28    False
2021-11-06    False
2021-11-17    False
2021-11-19    False
2021-11-23    False
2021-12-11    False
2021-12-18    False
2021-12-26    False
Name: Loss_1, dtype: bool